# COMPASS Engine: Interactive Demo

### Clinical Orchestrated Multi-modal Predictive Agentic Support System

Welcome to the **COMPASS** interactive demo. This notebook demonstrates the system's reasoning capabilities using synthetic clinical data.

**Objective**: To diagnose complex neuropsychiatric conditions by orchestrating multiple AI agents.

**Key Features**:
- **Dynamic Phenotype Targeting**: Select a specific diagnosis to investigate.
- **Transparent Reasoning**: View the step-by-step thoughts of the Orchestrator, Tools, and Predictor.
- **Multi-Agent Collaboration**: Orchestrator (Plan) -> Executor (Do) -> Integrator (Fuse) -> Predictor (Solve).

In [ ]:
import sys
import json
import ipywidgets as widgets
from IPython.display import display, Markdown, JSON, clear_output
from pathlib import Path
import subprocess
import time

# --- PATH SETUP ---
# We are in /utils. Navigate up two levels to project root (INFERENCE_PIPELINE)
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"[*] Project Root: {project_root}")

from multi_agent_system.main import run_compass_pipeline
from multi_agent_system.config.settings import get_settings, LLMBackend

# Locate Data
PSEUDO_DATA_ROOT = project_root / "multi_agent_system" / "data" / "pseudo_data"
if not PSEUDO_DATA_ROOT.exists():
    # Fallback to main data dir if pseudo_data is missing
    PSEUDO_DATA_ROOT = project_root / "data" / "__FEATURES__" / "COMPASS_data"

print(f"✅ COMPASS Engine Loaded. Data Source: {PSEUDO_DATA_ROOT.name}")

## 🚀 Quick Launch: Web Dashboard

Want the full visual experience? Click below to launch the **Live Dashboard** in your browser.
This runs the separate UI server (`main.py --ui`).

In [ ]:
def launch_dashboard(b):
    clear_output()
    print("🚀 Launching Dashboard on port 5005...")
    print("Go to: http://127.0.0.1:5005")
    try:
        # Use sys.executable to ensure we use the same python env
        main_script = project_root / "multi_agent_system" / "main.py"
        subprocess.Popen([sys.executable, str(main_script), "--ui"])
    except Exception as e:
        print(f"Error launching: {e}")

btn = widgets.Button(description="Launch Web Dashboard", button_style='info', icon='rocket')
btn.on_click(launch_dashboard)
display(btn)

## 🔬 Interactive Notebook Execution

Alternatively, run the analysis directly here in the notebook cells.

### 1. Configuration
Select a subject and your target condition.

In [ ]:
# Find available subjects
subjects = [d.name for d in PSEUDO_DATA_ROOT.iterdir() if d.is_dir() and not d.name.startswith('.')]
subjects.sort()

# Interactive Widgets
subject_dropdown = widgets.Dropdown(
    options=subjects,
    description='Subject:',
)

target_input = widgets.Text(
    value='Major Depressive Disorder',
    description='Target Pattern:',
    placeholder='e.g. Anxiety'
)

# Backend Selection
backend_dropdown = widgets.Dropdown(
    options=[('OpenAI API (GPT-5)', 'openai'), ('Local LLM (Open Source)', 'local')],
    value='openai',
    description='Backend:',
)

model_input = widgets.Text(
    value='Qwen/Qwen2.5-0.5B-Instruct',
    description='Local Model:',
    disabled=True
)

def on_backend_change(change):
    if change['new'] == 'local':
        model_input.disabled = False
    else:
        model_input.disabled = True

backend_dropdown.observe(on_backend_change, names='value')

display(subject_dropdown, target_input, backend_dropdown, model_input)

### 2. Run Analysis
This executes the pipeline. Output is verbose to show the agent's reasoning.

In [ ]:
PARTICIPANT_DIR = PSEUDO_DATA_ROOT / subject_dropdown.value
TARGET = target_input.value

print(f"Selected Participant: {PARTICIPANT_DIR}")

# Apply Backend Settings
settings = get_settings()
if backend_dropdown.value == 'local':
    settings.models.backend = LLMBackend.LOCAL
    settings.models.local_model_name = model_input.value
    print(f"⚙️ Configured for LOCAL inference using {model_input.value}")
else:
    settings.models.backend = LLMBackend.OPENAI
    print("⚙️ Configured for OPENAI inference")

print(f"🚀 STARTING ANALYSIS for: {TARGET}...")

# Run Pipeline
try:
    result = run_compass_pipeline(
        participant_dir=PARTICIPANT_DIR,
        target_condition=TARGET,
        max_iterations=2,
        verbose=True,
        interactive_ui=False # Use CLI output mode for notebook compatibility
    )
except Exception as e:
    print(f"❌ Execution Error: {e}")
    result = None

### 3. Inspector
View the structured results and decision trace.

In [ ]:
if result:
    log_path = Path(result['output_dir']) / f"execution_log_{result['participant_id']}.json"
    report_path = Path(result['output_dir']) / f"report_{result['participant_id']}.md"

    print(f"\n📊 RESULT: {result['prediction']} ({result['probability']:.1%})")
    
    if report_path.exists():
        display(Markdown("### Final Clinical Report"))
        with open(report_path, 'r') as f:
            display(Markdown(f.read()))
else:
    print("No successful result to inspect.")